In [1]:
# This cell imports relevant libraries
import pandas as pd
import geopandas as gpd
import os

In [2]:
# CUSTOMIZE THIS CELL USING STATE'S IDENTIFIERS
# This cell prompts users for their state of interest's identifiers
print("Input state abbreviation (e.g. ca):")
this_state_abbr = input()
print("Input state FIPS code (e.g. 06):")
this_state_fips = input()

Input state abbreviation (e.g. ca):
wa
Input state FIPS code (e.g. 06):
53


In [3]:
# This function consolidates a state's identifiers
def state_dict(abbreviation, fips_code):
    return dict([('abbr', abbreviation), ('fips', fips_code)])

In [4]:
# CUSTOMIZE THIS CELL USING STATE'S IDENTIFIERS
# This cell consolidates the user's state of interest's identifiers
this_state = state_dict(this_state_abbr, this_state_fips)

In [5]:
# This cell reads in the data obtained from various census databases
bgroup_file_name = "tl_2019_" + this_state[\
    'fips'] + "_bg/tl_2019_" + this_state[\
    'fips'] + "_bg.shp"
bgroups_shapefile = gpd.read_file(bgroup_file_name)
hh_file_name = this_state[\
    'abbr'] + '_bg_hh_acs_15_19.csv'
bgroups_hh_acs_15_19 = pd.read_csv(hh_file_name)

In [6]:
# This cell cleans and combines the block group data
fips_prefix = int(this_state['fips']) * 10000000000
num_bgroups = len(bgroups_hh_acs_15_19)
pd.options.mode.chained_assignment = None
bgroups_hh_acs_15_19['[["State_name"'] = bgroups_hh_acs_15_19[\
    '[["State_name"'].str[2:-1]
bgroups_hh_acs_15_19['block group]'] = bgroups_hh_acs_15_19[\
    'block group]'].str[:-1]
bgroups_hh_acs_15_19['block group]'][\
    num_bgroups - 1] = bgroups_hh_acs_15_19[\
    'block group]'][num_bgroups - 1].strip(']')
bgroups_hh_acs_15_19 = bgroups_hh_acs_15_19.rename(\
    columns = {'block group]': 'bgroup', \
    'Tot_Housing_Units_ACS_15_19': 'households'})
bgroups_hh_acs_15_19 = bgroups_hh_acs_15_19.drop(\
    columns = ['[["State_name"', 'County_name', 'Unnamed: 7'])
bgroups_hh_acs_15_19 = bgroups_hh_acs_15_19.dropna()
bgroups_hh_acs_15_19['GEOID'] = int(fips_prefix) + \
    10000000 * bgroups_hh_acs_15_19['county'].astype(int) + \
    10 * bgroups_hh_acs_15_19['tract'].astype(int) + \
    bgroups_hh_acs_15_19['bgroup'].astype(int)
bgroups_shapefile['GEOID'] = bgroups_shapefile['GEOID'].astype(int)
bgroups_shapefile = bgroups_shapefile.drop(\
    columns = ['STATEFP', 'COUNTYFP', \
    'TRACTCE', 'BLKGRPCE'])
bgroups_combo = pd.merge(\
    bgroups_shapefile, bgroups_hh_acs_15_19, \
    how = 'inner', on = 'GEOID')
bgroups_combo['GEOID'] = bgroups_combo['GEOID'].astype(str)
if len(bgroups_combo['GEOID'][0]) == 11:
    bgroups_combo['GEOID'] = bgroups_combo['GEOID'].apply(\
    lambda x: str(0) + x)
bgroups_combo['ALAND'] = bgroups_combo['ALAND'] / 1000000
bgroups_combo['HH_DENSITY'] = bgroups_combo[\
    'households'].astype(int) / bgroups_combo['ALAND']
bgroups_combo = bgroups_combo.sort_values(\
    'HH_DENSITY', ascending = False)

In [7]:
import os.path
from os import path
export_folder_name = this_state['abbr'] + "_bgroups"
if not path.exists(export_folder_name):
    os.makedirs(export_folder_name)
    export_file_name = this_state['abbr'] + "_bgroups.shp"
    bgroups_combo.to_file(\
        driver = 'ESRI Shapefile', \
        filename = export_folder_name + "/" + export_file_name)
else:
    print(export_folder_name + " already exists")